PA2 Notebook 2: Analysis
===

This is the second notebook for DRWI Pollution Assessment Stage 2 (PA2) analysis.

It reads dataframes prepared in Notebook 1 and calculates Pollution Assessment metrics necessary for the Stage 2 Assessment.

# Installation and Setup

Carefully follow our **[Installation Instructions](README.md#get-started)**, especially including:
- Creating a virtual environment for this repository (step 3)

## Import Python Dependencies

In [1]:
from pathlib import Path
from importlib import reload

import pandas    as pd
import geopandas as gpd

# geo packages
from shapely.geometry import Polygon
import contextily as ctx

# packages for viz 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from  matplotlib.colors import LogNorm

In [4]:
# Confirm GeoPandas >= 0.11, for full GeoParquet support
print("Geopandas: ", gpd.__version__)

Geopandas:  0.11.1


In [2]:
# Confirm that this repo is in your Python Path
!conda-develop /Users/aaufdenkampe/Documents/Python/pollution-assessment/src

path exists, skipping /Users/aaufdenkampe/Documents/Python/pollution-assessment/src
completed operation for: /Users/aaufdenkampe/Documents/Python/pollution-assessment/src


In [3]:
# Custom functions for Pollution Assessment
import pollution_assessment as pa
# import pollution_assessment.calc
# import pollution_assessment.plot

In [4]:
pa.__version__

'0.1.0'

In [5]:
reload(pa)

<module 'pollution_assessment' from '/Users/aaufdenkampe/Documents/Python/pollution-assessment/src/pollution_assessment/__init__.py'>

## Set Paths


In [5]:
# Set your project directory to your local folder for your clone of this repository
project_path = Path.cwd().parent
project_path

PosixPath('/Users/aaufdenkampe/Documents/Python/pollution-assessment')

In [6]:
# Assign relative paths for the data OUTPUT folder.
pa2_data_output_folder = Path('stage2/data_output')

data_output_path = project_path /pa2_data_output_folder

## Open Files from Notebook 1

In [7]:
%%time
# read geometry data from GeoParquet files
reach_gdf = gpd.read_parquet(data_output_path /'reach_gdf.parquet')
catch_gdf = gpd.read_parquet(data_output_path /'catch_gdf.parquet')

CPU times: user 1.13 s, sys: 119 ms, total: 1.24 s
Wall time: 1.23 s


In [8]:
%%time
# read results data from Parquet files
reach_concs_df = pd.read_parquet(data_output_path /'reach_concs_df.parquet')
catch_loads_df = pd.read_parquet(data_output_path /'catch_loads_df.parquet')

CPU times: user 79.8 ms, sys: 16.2 ms, total: 96 ms
Wall time: 41.6 ms


## Open Files from FieldDOC

In [9]:
# Practices
rest_gdf = gpd.read_parquet(project_path / 'stage2/private/restoration_bmps_from_FieldDoc.parquet')
prot_gdf = gpd.read_parquet(project_path / 'stage2/private/protection_bmps_from_FieldDoc.parquet')

# Practices by COMID
rest_comid_gdf = gpd.read_parquet(project_path / 'stage2/private/restoration_df.parquet')
prot_comid_gdf = gpd.read_parquet(project_path / 'stage2/private/protection_df.parquet')

In [38]:
rest_gdf

,practice_name,practice_id,program_name,program_id,organization,description,practice_type,created_at,modified_at,tn,tp,tss,geometry,drainage_geometry
0,Stream Buffer,5147,Delaware River Restoration Fund,1,Pennsylvania Environmental Council,None,Forest Buffer - Narrow,2018-08-30T12:29:03.453690Z,2020-12-02T14:36:40.538741Z,0.12,0.01,6.38,"POLYGON ((-75.10082 40.07019, -75.10082 40.070...","POLYGON ((-75.09879 40.06976, -75.09877 40.069..."
1,Easement,5288,Delaware River Operational Fund,8,The Commons,None,Conservation easement,2019-01-04T18:42:01.861717Z,2019-11-15T20:56:34.050925Z,0.00,0.00,0.00,"POLYGON ((-75.76890 41.20180, -75.76906 41.200...","POLYGON ((-75.76517 41.20116, -75.76515 41.201..."
2,Manure Storage,5305,Delaware River Restoration Fund,1,National Fish and Wildlife Foundation,Waste Storage Facility (313) Dry Stack - 1800 ...,Waste Storage Facility,2019-01-15T16:28:44.575411Z,2021-08-10T13:48:27.561319Z,6.30,2.12,0.00,"POLYGON ((-75.74194 40.50499, -75.74177 40.505...","POLYGON ((-75.74105 40.50483, -75.74102 40.504..."
3,Constructed Wetland,5327,Delaware River Restoration Fund,1,Pennsylvania Environmental Council,225' of hard armoring was removed from a chann...,Constructed Wetland,2019-01-17T20:24:44.695328Z,2021-01-28T21:40:24.747522Z,14.32,5.42,24539.59,"POLYGON ((-75.10072 40.07047, -75.10082 40.070...","POLYGON ((-75.09846 40.07007, -75.09843 40.070..."
4,Stream Restoration,5329,Delaware River Restoration Fund,1,Pennsylvania Environmental Council,This is a 225' stream restoration project,Urban Stream Restoration,2019-01-17T20:57:41.756424Z,2021-05-06T20:54:30.207584Z,43.65,39.15,25875.00,"POLYGON ((-75.10079 40.07052, -75.10063 40.070...","POLYGON ((-75.09846 40.07007, -75.09843 40.070..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,Waste Transfer,56059,Delaware River Restoration Fund,1,The Commons,None,Waste Transfer,2022-03-31T13:59:20.214390Z,2022-03-31T14:00:08.515768Z,0.00,0.00,0.00,None,None
1164,Waste Transfer,56060,Delaware River Restoration Fund,1,The Commons,None,Waste Transfer,2022-03-31T14:03:53.231731Z,2022-03-31T14:04:08.067781Z,0.00,0.00,0.00,None,None
1165,Grassed Waterway,56061,Delaware River Restoration Fund,1,The Commons,None,Grassed Waterway,2022-03-31T14:06:46.931571Z,2022-03-31T14:07:40.531113Z,3.95,1.12,1004.60,"POLYGON ((-75.88527 40.06592, -75.88509 40.065...","POLYGON ((-75.88509 40.06587, -75.88506 40.065..."
1166,Conservation Planning,56062,Delaware River Restoration Fund,1,The Commons,None,Comprehensive Nutrient Management Plan,2022-03-31T14:11:24.078572Z,2022-03-31T14:12:08.907506Z,75.82,32.50,0.00,"POLYGON ((-75.86979 40.06946, -75.87084 40.070...","POLYGON ((-75.86968 40.06940, -75.86966 40.069..."


In [10]:
prot_gdf

,practice_name,practice_id,program_name,program_id,organization,description,practice_type,created_at,modified_at,tot_pwr,head_pwr,nat_land,ara_pwr,wet_pwr,dev_land,ag_land,str_bank,geometry
0,Bartolacci,5301,Delaware River Watershed Protection Fund - For...,5,Natural Lands Trust,Land Protection \n\nShapefile needs to be load...,Conservation easement,2019-01-14T20:29:48.955938Z,2020-12-21T18:28:07.158142Z,26.00,0.00,39.31,25.94,8.59,0.02,0.00,0.43,"POLYGON ((-75.22603 41.12858, -75.22736 41.128..."
1,Pocono Mountain Bluestone,5302,Delaware River Watershed Protection Fund - For...,5,Natural Lands Trust,Land Protection,Conservation easement,2019-01-14T20:35:20.889852Z,2021-07-19T13:56:40.756288Z,76.96,2.32,271.32,72.70,8.09,10.90,0.00,1.34,"MULTIPOLYGON (((-75.23886 41.11628, -75.23923 ..."
2,Easement Acquisition,5337,Delaware River Watershed Protection Fund - For...,5,The Commons,None,Conservation easement,2019-01-18T16:36:17.047255Z,2021-07-07T17:30:23.422991Z,98.35,98.35,16.46,32.17,0.00,1.33,80.73,0.66,"MULTIPOLYGON (((-75.60471 40.13618, -75.60281 ..."
3,Acquisition,5431,Delaware River Watershed Protection Fund - For...,5,The Land Conservancy of New Jersey,None,Fee acquisition,2019-01-25T16:48:57.618083Z,2021-03-25T18:38:01.119992Z,44.82,44.82,45.15,0.07,0.00,0.22,0.00,0.00,"POLYGON ((-74.72381 40.95739, -74.72329 40.957..."
4,Yards Creek Preserve Phase I - Subject,5436,Delaware River Watershed Protection Fund - For...,5,The Land Conservancy of New Jersey,None,Fee acquisition,2019-01-25T18:36:23.757416Z,2021-03-25T18:34:49.498465Z,113.52,113.52,114.78,71.13,22.75,0.22,0.00,1.34,"POLYGON ((-75.00325 41.01775, -75.00231 41.014..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Yards Creek Preserve Phase VI,55055,Delaware River Watershed Protection Fund - For...,5,The Land Conservancy of New Jersey,The project is located in the NJ Highlands Clu...,Fee acquisition,2022-01-05T17:29:46.294169Z,2022-01-10T22:12:13.134459Z,15.13,15.13,14.68,3.87,0.00,0.00,0.00,0.00,"POLYGON ((-75.00032 41.00509, -74.99976 41.004..."
71,Porters Lake Hunting & Fishing Club - Subject ...,55080,Delaware River Watershed Protection Fund - For...,5,Delaware Highlands Conservancy,Proposed conservation easement area for the gr...,Conservation easement,2022-01-10T17:29:30.832356Z,2022-01-19T15:41:57.085338Z,1928.49,1127.10,2781.94,1278.78,403.40,3.78,0.00,14.62,"MULTIPOLYGON (((-75.06903 41.24877, -75.06784 ..."
72,Mill Creek Fen - Subject - Easement,55287,Delaware River Watershed Protection Fund - For...,5,Pocono Heritage Land Trust,29 acres in Barrett Twp.,Conservation easement,2022-01-21T14:09:20.371728Z,2022-01-21T16:09:43.912661Z,23.62,0.00,33.14,22.88,4.31,0.00,0.00,0.58,"MULTIPOLYGON (((-75.26094 41.15974, -75.26233 ..."
73,Bear Creek: Pinchot Forest Addition,55397,Delaware River Watershed Protection Fund - For...,5,The Nature Conservancy,None,Fee acquisition,2022-01-27T16:55:22.172988Z,2022-08-10T16:58:55.658461Z,94.93,94.93,95.85,8.21,3.58,0.00,0.00,0.06,"POLYGON ((-75.65811 41.22828, -75.65789 41.219..."


In [11]:
prot_comid_gdf

huc12  \
comid     practice_id                 
2583191   54313        020401050101   
2583195   51275        020401050101   
          51282        020401050101   
          54313        020401050101   
2583199   51275        020401050101   
...                             ...   
932040258 42237        020401060201   
          42239        020401060201   
932040267 55080        020401040602   
932040274 42237        020401060201   
          42239        020401060201   

                                                           practice_name  \
comid     practice_id                                                      
2583191   54313        Culvers Brook Preserve II - Subject - Fee Acqu...   
2583195   51275        Culvers Brook Preserve - Reforested Area (to >...   
          51282            Culvers Brook Preserve - Active Forested Area   
          54313        Culvers Brook Preserve II - Subject - Fee Acqu...   
2583199   51275        Culvers Brook Preserve - Reforested Area (to >...   
...                                                                  ...   
932040258 42237        Lehigh River Headwaters Acquisition Project - ...   
          42239        Lehigh River Headwaters Acquisition Project - ...   
932040267 55080        Porters Lake Hunting & Fishing Club - Subject ...   
932040274 42237        Lehigh River Headwaters Acquisition Project - ...   
          42239        Lehigh River Headwaters Acquisition Project - ...   

                                                            program_name  \
comid     practice_id                                                      
2583191   54313        Delaware River Watershed Protection Fund - For...   
2583195   51275        Delaware River Watershed Protection Fund - For...   
          51282        Delaware River Watershed Protection Fund - For...   
          54313        Delaware River Watershed Protection Fund - For...   
2583199   51275        Delaware River Watershed Protection Fund - For...   
...                                                                  ...   
932040258 42237        Delaware River Watershed Protection Fund - For...   
          42239        Delaware River Watershed Protection Fund - For...   
932040267 55080        Delaware River Watershed Protection Fund - For...   
932040274 42237        Delaware River Watershed Protection Fund - For...   
          42239        Delaware River Watershed Protection Fund - For...   

                       program_id                        organization  \
comid     practice_id                                                   
2583191   54313                 5  New Jersey Conservation Foundation   
2583195   51275                 5  New Jersey Conservation Foundation   
          51282                 5  New Jersey Conservation Foundation   
          54313                 5  New Jersey Conservation Foundation   
2583199   51275                 5  New Jersey Conservation Foundation   
...                           ...                                 ...   
932040258 42237                 5                         The Commons   
          42239                 5                         The Commons   
932040267 55080                 5      Delaware Highlands Conservancy   
932040274 42237                 5                         The Commons   
          42239                 5                         The Commons   

                                                             description  \
comid     practice_id                                                      
2583191   54313                                                     None   
2583195   51275        The WRWMG plans to use its extensive watershed...   
          51282                                                     None   
          54313                                                     None   
2583199   51275        The WRWMG plans to use its extensive watershed...   
...                                                           

# Baseline Results
Create mappable GeoDataFrames (GDF) of results for "baseline" model runs (i.e. 'No restoration or protection')

In [12]:
# Use pollutant names dictionary
pa.calc.pollutants

{'TotalN': 'tn', 'TotalP': 'tp', 'Sediment': 'tss'}

In [13]:
# Create Stream Reach Concentrations GDF
reach_concs_gdf = pa.calc.join_results(
    'reach', reach_gdf, reach_concs_df, 
    pa.calc.run_groups[0], ps=False
)
# Add columns with previously used short names
for pollutant in pa.calc.pollutants.keys():
    reach_concs_gdf [f'{pa.calc.pollutants[pollutant]}_conc'] = (
        reach_concs_gdf[f'{pollutant}'])

# reach_concs_gdf.info()

In [14]:
# Create Catchment Loads GDF
catch_loads_gdf = pa.calc.join_results(
    'catch', catch_gdf, catch_loads_df, 
    pa.calc.run_groups[0], ps=False
)
pa.calc.add_loadrate(catch_loads_gdf)
catch_loads_gdf.head()

,catchment_hectares,watershed_hectares,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,Source,Sediment,TotalN,TotalP,run_group,funding_sources,with_attenuation,tn_loadrate,tp_loadrate,tss_loadrate
comid,,,,,,,,,,,,,,,,,,,,,
1748535,6496.7052,6501.69,43.699,"MULTIPOLYGON (((-8301340.781 5199034.787, -830...",drb,<NA>,74914,74914,020401020302,1,...,Total Local Load,1.101612e+06,12680.544786,1189.608231,No restoration or protection,NaN,True,1.951842,0.183109,169.564734
1748537,1663.1712,1664.46,11.189,"MULTIPOLYGON (((-8304909.314 5200051.727, -830...",drb,<NA>,74913,74913,020401020302,1,...,Total Local Load,2.013339e+05,3771.332143,363.366436,No restoration or protection,NaN,True,2.267555,0.218478,121.054237
1748539,1639.4128,1640.70,11.223,"MULTIPOLYGON (((-8315191.630 5191704.467, -831...",drb,<NA>,74921,74921,020401020305,1,...,Total Local Load,2.514024e+05,3133.430355,357.227990,No restoration or protection,NaN,True,1.911313,0.217900,153.349047
1748541,3013.8348,12912.30,86.528,"MULTIPOLYGON (((-8309824.403 5193427.492, -830...",drb,<NA>,74911,74915,020401020302,2,...,Total Local Load,6.177144e+05,6409.514442,668.969079,No restoration or protection,NaN,True,2.126697,0.221966,204.959595
1748543,1151.0990,5232.87,35.389,"MULTIPOLYGON (((-8312514.529 5185023.831, -831...",drb,<NA>,74920,74922,020401020305,2,...,Total Local Load,1.989546e+05,2918.236825,317.461447,No restoration or protection,NaN,True,2.535174,0.275790,172.838845


# XS: Excess Pollution Results

```
excess pollution = total pollution 
                   – threshold pollution target
```

In [15]:
# Open dictionary of Target Values
pa.calc.targets

{'tn': {'loadrate_target': 17.07, 'conc_target': 4.73},
 'tp': {'loadrate_target': 0.31, 'conc_target': 0.09},
 'tss': {'loadrate_target': 923.8, 'conc_target': 237.3}}

In [16]:
pa.calc.add_excess('reach', reach_concs_gdf)
reach_concs_gdf.head(3)

,catchment_hectares,watershed_hectares,maflowv,geom,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,TotalP,run_group,funding_sources,with_attenuation,tn_conc,tp_conc,tss_conc,tn_conc_xs,tp_conc_xs,tss_conc_xs
comid,,,,,,,,,,,,,,,,,,,,,
1748535,6496.7052,6501.69,43.699,MULTILINESTRING Z ((-8295323.930 5214456.622 0...,drb,<NA>,74914,74914,020401020302,1,...,0.030464,No restoration or protection,NaN,True,0.324727,0.030464,28.210388,-4.405273,-0.059536,-209.089612
1748537,1663.1712,1664.46,11.189,MULTILINESTRING Z ((-8304623.226 5207684.737 0...,drb,<NA>,74913,74913,020401020302,1,...,0.036342,No restoration or protection,NaN,True,0.377186,0.036342,20.136201,-4.352814,-0.053658,-217.163799
1748539,1639.4128,1640.70,11.223,MULTILINESTRING Z ((-8316446.558 5197994.113 0...,drb,<NA>,74921,74921,020401020305,1,...,0.035620,No restoration or protection,NaN,True,0.312437,0.035620,25.067574,-4.417563,-0.054380,-212.232426


In [17]:
pa.calc.add_excess('catch', catch_loads_gdf)
catch_loads_gdf.head(3)

,catchment_hectares,watershed_hectares,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,TotalP,run_group,funding_sources,with_attenuation,tn_loadrate,tp_loadrate,tss_loadrate,tn_loadrate_xs,tp_loadrate_xs,tss_loadrate_xs
comid,,,,,,,,,,,,,,,,,,,,,
1748535,6496.7052,6501.69,43.699,"MULTIPOLYGON (((-8301340.781 5199034.787, -830...",drb,<NA>,74914,74914,020401020302,1,...,1189.608231,No restoration or protection,NaN,True,1.951842,0.183109,169.564734,-15.118158,-0.126891,-754.235266
1748537,1663.1712,1664.46,11.189,"MULTIPOLYGON (((-8304909.314 5200051.727, -830...",drb,<NA>,74913,74913,020401020302,1,...,363.366436,No restoration or protection,NaN,True,2.267555,0.218478,121.054237,-14.802445,-0.091522,-802.745763
1748539,1639.4128,1640.70,11.223,"MULTIPOLYGON (((-8315191.630 5191704.467, -831...",drb,<NA>,74921,74921,020401020305,1,...,357.227990,No restoration or protection,NaN,True,1.911313,0.217900,153.349047,-15.158687,-0.092100,-770.450953


# XSNPS: Excess Non-Point Source Results

```
excess nonpoint source pollution = excess pollution 
                                   – point source pollution
```

In [18]:
pa.calc.add_xsnps('reach', reach_concs_gdf, reach_concs_df)
reach_concs_gdf.head(3)

,catchment_hectares,watershed_hectares,maflowv,geom,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,with_attenuation,tn_conc,tp_conc,tss_conc,tn_conc_xs,tp_conc_xs,tss_conc_xs,tn_conc_xsnps,tp_conc_xsnps,tss_conc_xsnps
comid,,,,,,,,,,,,,,,,,,,,,
1748535,6496.7052,6501.69,43.699,MULTILINESTRING Z ((-8295323.930 5214456.622 0...,drb,<NA>,74914,74914,020401020302,1,...,True,0.324727,0.030464,28.210388,-4.405273,-0.059536,-209.089612,-4.405273,-0.059536,-209.089612
1748537,1663.1712,1664.46,11.189,MULTILINESTRING Z ((-8304623.226 5207684.737 0...,drb,<NA>,74913,74913,020401020302,1,...,True,0.377186,0.036342,20.136201,-4.352814,-0.053658,-217.163799,-4.352814,-0.053658,-217.163799
1748539,1639.4128,1640.70,11.223,MULTILINESTRING Z ((-8316446.558 5197994.113 0...,drb,<NA>,74921,74921,020401020305,1,...,True,0.312437,0.035620,25.067574,-4.417563,-0.054380,-212.232426,-4.417563,-0.054380,-212.232426


In [19]:
pa.calc.add_xsnps('catch', catch_loads_gdf, catch_loads_df)
catch_loads_gdf.head(3)

,catchment_hectares,watershed_hectares,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,with_attenuation,tn_loadrate,tp_loadrate,tss_loadrate,tn_loadrate_xs,tp_loadrate_xs,tss_loadrate_xs,tn_loadrate_xsnps,tp_loadrate_xsnps,tss_loadrate_xsnps
comid,,,,,,,,,,,,,,,,,,,,,
1748535,6496.7052,6501.69,43.699,"MULTIPOLYGON (((-8301340.781 5199034.787, -830...",drb,<NA>,74914,74914,020401020302,1,...,True,1.951842,0.183109,169.564734,-15.118158,-0.126891,-754.235266,-15.118158,-0.126891,-754.235266
1748537,1663.1712,1664.46,11.189,"MULTIPOLYGON (((-8304909.314 5200051.727, -830...",drb,<NA>,74913,74913,020401020302,1,...,True,2.267555,0.218478,121.054237,-14.802445,-0.091522,-802.745763,-14.802445,-0.091522,-802.745763
1748539,1639.4128,1640.70,11.223,"MULTIPOLYGON (((-8315191.630 5191704.467, -831...",drb,<NA>,74921,74921,020401020305,1,...,True,1.911313,0.217900,153.349047,-15.158687,-0.092100,-770.450953,-15.158687,-0.092100,-770.450953


## Confirm Results vs. MMW Sub-basin

Confirm values make sense for COMIDs with:
- no point sources: `4648450`
- large point sources: `932040160`

In [20]:
reach_concs_gdf[['tp_conc',
                'tp_conc_xs',
                'tp_conc_xsnps',
               ]].loc[[4648450, 932040160]]

,tp_conc,tp_conc_xs,tp_conc_xsnps
comid,,,
4648450,0.324502,0.234502,0.234502
932040160,0.284033,0.194033,0.107405


In [21]:
catch_loads_gdf[['tp_loadrate',
                'tp_loadrate_xs',
                'tp_loadrate_xsnps',
               ]].loc[[4648450, 932040160]]

,tp_loadrate,tp_loadrate_xs,tp_loadrate_xsnps
comid,,,
4648450,1.554258,1.244258,1.244258
932040160,7.054176,6.744176,0.187592


In [22]:
catch_loads_gdf[['tn_loadrate',
                'tn_loadrate_xs',
                'tn_loadrate_xsnps',
               ]].loc[[4648450, 932040160]]

,tn_loadrate,tn_loadrate_xs,tn_loadrate_xsnps
comid,,,
4648450,25.356569,8.286569,8.286569
932040160,49.770205,32.700205,-8.232529


In [23]:
df = pa.calc.calc_loadrate(catch_loads_gdf, catch_loads_df, 
                            'TotalN', 
                            pa.calc.run_groups[0], ps=True)
df.loc[[4648450, 932040160]]

comid
4648450       0.000000
932040160    40.932734
dtype: float64

# REM: Remaining XSNPS after Restoration

In [24]:
for group_key in [1,2,3]:
    pa.calc.add_remaining('reach', reach_concs_gdf, reach_concs_df, group_key)
reach_concs_gdf.loc[[4648450, 932040160]]

,catchment_hectares,watershed_hectares,maflowv,geom,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,tss_conc_xsnps,tn_conc_rem1,tp_conc_rem1,tss_conc_rem1,tn_conc_rem2,tp_conc_rem2,tss_conc_rem2,tn_conc_rem3,tp_conc_rem3,tss_conc_rem3
comid,,,,,,,,,,,,,,,,,,,,,
4648450,263.4373,263.61,1.412,MULTILINESTRING Z ((-8449613.219 4882948.059 0...,Brandywine and Christina,<NA>,64639,64639,020402050202,1,...,-86.572683,0.249977,0.062610,-112.257291,0.249977,0.062610,-112.257291,0.248132,0.059352,-112.762403
932040160,497.8113,14124.96,74.903,MULTILINESTRING Z ((-8440707.897 4862261.408 0...,Brandywine and Christina,<NA>,64609,64658,020402050202,3,...,-96.816710,-0.596242,0.056908,-193.251167,-0.596242,0.056908,-193.251167,-0.597487,0.055151,-193.573524


In [25]:
for group_key in [1,2,3]:
    pa.calc.add_remaining('catch', catch_loads_gdf, catch_loads_df, group_key)
catch_loads_gdf.loc[[4648450, 932040160]]

,catchment_hectares,watershed_hectares,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,tss_loadrate_xsnps,tn_loadrate_rem1,tp_loadrate_rem1,tss_loadrate_rem1,tn_loadrate_rem2,tp_loadrate_rem2,tss_loadrate_rem2,tn_loadrate_rem3,tp_loadrate_rem3,tss_loadrate_rem3
comid,,,,,,,,,,,,,,,,,,,,,
4648450,263.4373,263.61,1.412,"MULTIPOLYGON (((-8449229.677 4880762.406, -844...",Brandywine and Christina,<NA>,64639,64639,020402050202,1,...,-201.865928,6.782443,0.420951,-324.886720,6.782443,0.420951,-324.886720,6.773609,0.405346,-327.306039
932040160,497.8113,14124.96,74.903,"MULTIPOLYGON (((-8442711.700 4860467.546, -844...",Brandywine and Christina,<NA>,64609,64658,020402050202,3,...,162.418203,-8.232529,0.187592,162.418203,-8.232529,0.187592,162.418203,-8.233741,0.186018,162.114027


# AVOID: Avoided Loads due to Protection

In [26]:
pa.calc.run_groups[4]

'Direct WPF Protection'

In [29]:
pa.calc.add_avoided('reach', reach_concs_gdf, reach_concs_df, 4)
reach_concs_gdf.loc[pa.calc.comid_test_dict.keys(),
    ['tp_conc', 'tp_conc_xsnps','tp_conc_rem3', 'tp_conc_avoid']]

,tp_conc,tp_conc_xsnps,tp_conc_rem3,tp_conc_avoid
comid,,,,
4648450,0.324502,0.234502,0.059352,0.000000
4648684,0.614707,0.152823,0.147136,0.000000
932040160,0.284033,0.107405,0.055151,0.000000
2583195,0.064066,-0.025934,-0.025940,0.001256
932040230,0.006167,-0.083833,-0.083833,0.000514


In [30]:
pa.calc.add_avoided('catch', catch_loads_gdf, catch_loads_df, 4)
catch_loads_gdf.loc[pa.calc.comid_test_dict.keys(),
    ['tp_loadrate', 'tp_loadrate_xsnps','tp_loadrate_rem3', 'tp_loadrate_avoid']]

,tp_loadrate,tp_loadrate_xsnps,tp_loadrate_rem3,tp_loadrate_avoid
comid,,,,
4648450,1.554258,1.244258,0.405346,0.000000
4648684,5.058250,0.877691,0.860502,0.000000
932040160,7.054176,0.187592,0.186018,0.000000
2583195,0.359205,0.049205,0.049172,0.007041
932040230,0.082214,-0.227786,-0.227786,0.007001


In [31]:
catch_loads_gdf.loc[(catch_loads_gdf.tp_loadrate_avoid > 0)]

,catchment_hectares,watershed_hectares,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,tss_loadrate_rem1,tn_loadrate_rem2,tp_loadrate_rem2,tss_loadrate_rem2,tn_loadrate_rem3,tp_loadrate_rem3,tss_loadrate_rem3,tn_loadrate_avoid,tp_loadrate_avoid,tss_loadrate_avoid
comid,,,,,,,,,,,,,,,,,,,,,
2583191,277.9706,356.76,2.257,"MULTIPOLYGON (((-8321373.063 5036691.907, -832...",New Jersey Highlands,<NA>,70084,70085,020401050101,1,...,-645.705971,-14.543512,-0.070038,-645.705971,-14.543590,-0.070066,-645.721320,0.000034,0.000305,0.012710
2583195,244.9663,245.16,1.537,"MULTIPOLYGON (((-8322884.353 5034593.476, -832...",New Jersey Highlands,<NA>,70082,70082,020401050101,1,...,-492.602728,-13.993447,0.049205,-492.602728,-13.993536,0.049172,-492.620739,0.000776,0.007041,0.292851
2583199,215.1096,1018.44,7.164,"MULTIPOLYGON (((-8325931.539 5035114.436, -832...",New Jersey Highlands,<NA>,70074,70080,020401050101,2,...,-444.071676,-15.198850,-0.088779,-444.071676,-15.199058,-0.088856,-444.112843,0.000358,0.003214,0.134492
2583463,163.6690,163.80,1.131,"MULTIPOLYGON (((-8348195.342 5014278.655, -834...",New Jersey Highlands,<NA>,70175,70175,020401050104,1,...,-733.990121,-16.267772,-0.227914,-733.990121,-16.267783,-0.227928,-733.990889,0.000018,0.000164,0.007471
2583501,343.6150,1712.34,13.092,"MULTIPOLYGON (((-8349076.358 5012570.234, -834...",New Jersey Highlands,<NA>,70168,70175,020401050104,2,...,-762.747681,-16.007439,-0.221786,-762.747681,-16.007477,-0.221847,-762.749445,0.000030,0.000273,0.012480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932040230,21.4928,667.08,5.065,"MULTIPOLYGON (((-8357072.635 5048834.448, -835...",Poconos and Kittatinny,<NA>,70857,70860,020401040602,1,...,-711.287136,-16.552876,-0.227786,-711.287136,-16.552876,-0.227786,-711.287136,0.000791,0.007001,0.327289
932040231,271.7625,368.64,2.541,"MULTIPOLYGON (((-8354173.968 5049819.487, -835...",Poconos and Kittatinny,<NA>,70859,70860,020401040602,1,...,-847.183465,-16.630928,-0.270726,-847.183465,-16.630928,-0.270726,-847.183465,0.000296,0.002561,0.121161
932040232,276.7087,645.57,4.902,"MULTIPOLYGON (((-8356477.168 5048011.609, -835...",Poconos and Kittatinny,<NA>,70858,70860,020401040602,1,...,-828.849766,-16.075971,-0.255903,-828.849766,-16.075972,-0.255904,-828.849952,0.000734,0.006189,0.297029


In [3]:
# Reload module to apply any new edits to functions
reload(pa.calc)

NameError: name 'pa' is not defined

# Confirm Results

In [33]:
pa.calc.comid_test_dict

{4648450: 'no point sources',
 4648684: 'Upper E Branch Brandywine',
 932040160: 'large point sources',
 2583195: 'protection projects',
 932040230: 'restoration and protection projects'}

In [36]:
catch_loads_gdf[['tp_loadrate',
                'tp_loadrate_xs',
                'tp_loadrate_xsnps',
                'tp_loadrate_rem1',
                'tp_loadrate_rem2',
                'tp_loadrate_rem3',
                'tp_loadrate_avoid',
               ]].loc[pa.calc.comid_test_dict.keys()]

,tp_loadrate,tp_loadrate_xs,tp_loadrate_xsnps,tp_loadrate_rem1,tp_loadrate_rem2,tp_loadrate_rem3,tp_loadrate_avoid
comid,,,,,,,
4648450,1.554258,1.244258,1.244258,0.420951,0.420951,0.405346,0.000000
4648684,5.058250,4.748250,0.877691,0.873814,0.873814,0.860502,0.000000
932040160,7.054176,6.744176,0.187592,0.187592,0.187592,0.186018,0.000000
2583195,0.359205,0.049205,0.049205,0.049205,0.049205,0.049172,0.007041
932040230,0.082214,-0.227786,-0.227786,-0.227786,-0.227786,-0.227786,0.007001


In [35]:
catch_loads_gdf

,catchment_hectares,watershed_hectares,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,tss_loadrate_rem1,tn_loadrate_rem2,tp_loadrate_rem2,tss_loadrate_rem2,tn_loadrate_rem3,tp_loadrate_rem3,tss_loadrate_rem3,tn_loadrate_avoid,tp_loadrate_avoid,tss_loadrate_avoid
comid,,,,,,,,,,,,,,,,,,,,,
1748535,6496.7052,6501.69,43.699,"MULTIPOLYGON (((-8301340.781 5199034.787, -830...",drb,<NA>,74914,74914,020401020302,1,...,-754.235266,-15.118158,-0.126891,-754.235266,-15.118159,-0.126891,-754.235502,0.0,0.0,0.0
1748537,1663.1712,1664.46,11.189,"MULTIPOLYGON (((-8304909.314 5200051.727, -830...",drb,<NA>,74913,74913,020401020302,1,...,-802.745763,-14.802445,-0.091522,-802.745763,-14.802446,-0.091522,-802.746034,0.0,0.0,0.0
1748539,1639.4128,1640.70,11.223,"MULTIPOLYGON (((-8315191.630 5191704.467, -831...",drb,<NA>,74921,74921,020401020305,1,...,-770.450953,-15.158687,-0.092100,-770.450953,-15.158689,-0.092100,-770.451212,0.0,0.0,0.0
1748541,3013.8348,12912.30,86.528,"MULTIPOLYGON (((-8309824.403 5193427.492, -830...",drb,<NA>,74911,74915,020401020302,2,...,-718.840405,-14.943303,-0.088034,-718.840405,-14.943305,-0.088035,-718.840895,0.0,0.0,0.0
1748543,1151.0990,5232.87,35.389,"MULTIPOLYGON (((-8312514.529 5185023.831, -831...",drb,<NA>,74920,74922,020401020305,2,...,-750.961155,-14.534826,-0.034210,-750.961155,-14.534826,-0.034210,-750.961313,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932040366,2124.7248,2720941.47,17802.923,"MULTIPOLYGON (((-8403944.327 4826463.781, -840...",drb,<NA>,65070,76964,020402060103,7,...,-253.443080,-11.311054,-0.009946,-253.443080,-11.311461,-0.010079,-253.511418,0.0,0.0,0.0
932040367,788.7859,2717821.26,17788.281,"MULTIPOLYGON (((-8400739.269 4832000.931, -840...",drb,<NA>,65079,76964,020402060103,7,...,-390.234005,-9.776384,0.081706,-390.234005,-9.777731,0.081219,-390.461603,0.0,0.0,0.0
932040368,265.0275,2716120.08,17780.448,"MULTIPOLYGON (((-8399608.027 4833463.133, -839...",drb,<NA>,65080,76960,020402060103,7,...,-640.096341,-16.357360,-0.238284,-640.096341,-16.357360,-0.238284,-640.096341,0.0,0.0,0.0


# Save Calculated PA2 Results

In [37]:
%%time
# Save PA2 combined and calculated results
reach_concs_gdf.to_parquet(data_output_path /'reach_concs_gdf.parquet',compression='gzip')
catch_loads_gdf.to_parquet(data_output_path /'catch_loads_gdf.parquet',compression='gzip')

CPU times: user 4.47 s, sys: 92 ms, total: 4.57 s
Wall time: 4.81 s
